# Create a Web Application for an ETF Analyzer

In this Challenge assignment, you’ll build a financial database and web application by using SQL, Python, and the Voilà library to analyze the performance of a hypothetical fintech ETF.

Instructions: 

Use this notebook to complete your analysis of a fintech ETF that consists of four stocks: GOST, GS, PYPL, and SQ. Each stock has its own table in the `etf.db` database, which the `Starter_Code` folder also contains.

Analyze the daily returns of the ETF stocks both individually and as a whole. Then deploy the visualizations to a web application by using the Voilà library.

The detailed instructions are divided into the following parts:

* Analyze a single asset in the ETF

* Optimize data access with Advanced SQL queries

* Analyze the ETF portfolio

* Deploy the notebook as a web application

#### Analyze a Single Asset in the ETF

For this part of the assignment, you’ll use SQL queries with Python, Pandas, and hvPlot to analyze the performance of a single asset from the ETF.

Complete the following steps:

1. Write a SQL `SELECT` statement by using an f-string that reads all the PYPL data from the database. Using the SQL `SELECT` statement, execute a query that reads the PYPL data from the database into a Pandas DataFrame.

2. Use the `head` and `tail` functions to review the first five and the last five rows of the DataFrame. Make a note of the beginning and end dates that are available from this dataset. You’ll use this information to complete your analysis.

3. Using hvPlot, create an interactive visualization for the PYPL daily returns. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.

4. Using hvPlot, create an interactive visualization for the PYPL cumulative returns. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.

#### Optimize Data Access with Advanced SQL Queries

For this part of the assignment, you’ll continue to analyze a single asset (PYPL) from the ETF. You’ll use advanced SQL queries to optimize the efficiency of accessing data from the database.

Complete the following steps:

1. Access the closing prices for PYPL that are greater than 200 by completing the following steps:

    - Write a SQL `SELECT` statement to select the dates where the PYPL closing price was higher than 200.0.

    - Using the SQL statement, read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.

    - Select the “time” and “close” columns for those dates where the closing price was higher than 200.0.

2. Find the top 10 daily returns for PYPL by completing the following steps:

    -  Write a SQL statement to find the top 10 PYPL daily returns. Make sure to do the following:

        * Use `SELECT` to select only the “time” and “daily_returns” columns.

        * Use `ORDER` to sort the results in descending order by the “daily_returns” column.

        * Use `LIMIT` to limit the results to the top 10 daily return values.

    - Using the SQL statement, read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.

#### Analyze the ETF Portfolio

For this part of the assignment, you’ll build the entire ETF portfolio and then evaluate its performance. To do so, you’ll build the ETF portfolio by using SQL joins to combine all the data for each asset.

Complete the following steps:

1. Write a SQL query to join each table in the portfolio into a single DataFrame. To do so, complete the following steps:

    - Use a SQL inner join to join each table on the “time” column. Access the “time” column in the `GDOT` table via the `GDOT.time` syntax. Access the “time” columns from the other tables via similar syntax.

    - Using the SQL query, read the data from the database into a Pandas DataFrame. Review the resulting DataFrame.

2. Create a DataFrame that averages the “daily_returns” columns for all four assets. Review the resulting DataFrame.

    > **Hint** Assuming that this ETF contains equally weighted returns, you can average the returns for each asset to get the average returns of the portfolio. You can then use the average returns of the portfolio to calculate the annualized returns and the cumulative returns. For the calculation to get the average daily returns for the portfolio, use the following code:
    >
    > ```python
    > etf_portfolio_returns = etf_portfolio['daily_returns'].mean(axis=1)
    > ```
    >
    > You can use the average daily returns of the portfolio the same way that you used the daily returns of a single asset.

3. Use the average daily returns in the `etf_portfolio_returns` DataFrame to calculate the annualized returns for the portfolio. Display the annualized return value of the ETF portfolio.

> **Hint**  To calculate the annualized returns, multiply the mean of the `etf_portfolio_returns` values by 252.
>
> To convert the decimal values to percentages, multiply the results by 100.

4. Use the average daily returns in the `etf_portfolio_returns` DataFrame to calculate the cumulative returns of the ETF portfolio.

5. Using hvPlot, create an interactive line plot that visualizes the cumulative return values of the ETF portfolio. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.

#### Deploy the Notebook as a Web Application

For this part of the assignment, complete the following steps:

1. Use the Voilà library to deploy your notebook as a web application. You can deploy the web application locally on your computer.

2. Take a screen recording or screenshots to show how the web application appears when using Voilà. Include the recording or screenshots in the `README.md` file for your GitHub repository.


## Review the following code which imports the required libraries, initiates your SQLite database, popluates the database with records from the `etf.db` seed file that was included in your Starter_Code folder, creates the database engine, and confirms that data tables that it now contains.

In [184]:
# Importing the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
import sqlalchemy
import datetime
from datetime import date, datetime

# Create a temporary SQLite database and populate the database with content from the etf.db seed file
database_connection_string = 'sqlite:///etf.db'

# Create an engine to interact with the SQLite database
engine = sqlalchemy.create_engine(database_connection_string)

# Confirm that table names contained in the SQLite database.
engine.table_names()

/Users/Paola/Applications/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:16: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  app.launch_new_instance()


['GDOT', 'GS', 'PYPL', 'SQ']

In [336]:
def clean_date(date_string_df):
    clean_date=[]
    for row in range(0,len(date_string_df)):
        #print(date_string_df[row])
        clean_new=date_string_df[row][0:10]
        date_object = datetime.strptime(clean_new, "%Y-%m-%d")
        #print(date_string_df[row][0:10])
        #print(clean_new)
        clean_date.append(date_object)
        #clean_date
    return(clean_date)


## Analyze a single asset in the FinTech ETF

For this part of the assignment, you’ll use SQL queries with Python, Pandas, and hvPlot to analyze the performance of a single asset from the ETF.

Complete the following steps:

1. Write a SQL `SELECT` statement by using an f-string that reads all the PYPL data from the database. Using the SQL `SELECT` statement, execute a query that reads the PYPL data from the database into a Pandas DataFrame.

2. Use the `head` and `tail` functions to review the first five and the last five rows of the DataFrame. Make a note of the beginning and end dates that are available from this dataset. You’ll use this information to complete your analysis.

3. Using hvPlot, create an interactive visualization for the PYPL daily returns. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.

4. Using hvPlot, create an interactive visualization for the PYPL cumulative returns. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.



### Step 1: Write a SQL `SELECT` statement by using an f-string that reads all the PYPL data from the database. Using the SQL `SELECT` statement, execute a query that reads the PYPL data from the database into a Pandas DataFrame.

In [308]:
# Write a SQL query to SELECT all of the data from the PYPL table
query = """
SELECT * from PYPL
"""

# Use the query to read the PYPL data into a Pandas DataFrame
pypl_dataframe = pd.read_sql_query(query, con=engine)


### Step 2: Use the `head` and `tail` functions to review the first five and the last five rows of the DataFrame. Make a note of the beginning and end dates that are available from this dataset. You’ll use this information to complete your analysis.

In [309]:
# View the first 5 rows of the DataFrame.

display(pypl_dataframe.head())
display(pypl_dataframe.tail())



,time,open,high,low,close,volume,daily_returns
0,2016-12-16 00:00:00.000000,39.90,39.90,39.12,39.32,7298861,-0.005564
1,2016-12-19 00:00:00.000000,39.40,39.80,39.11,39.45,3436478,0.003306
2,2016-12-20 00:00:00.000000,39.61,39.74,39.26,39.74,2940991,0.007351
3,2016-12-21 00:00:00.000000,39.84,40.74,39.82,40.09,5826704,0.008807
4,2016-12-22 00:00:00.000000,40.04,40.09,39.54,39.68,4338385,-0.010227


,time,open,high,low,close,volume,daily_returns
994,2020-11-30 00:00:00.000000,212.51,215.83,207.0900,214.200,8992681,0.013629
995,2020-12-01 00:00:00.000000,217.15,220.57,214.3401,216.520,9148174,0.010831
996,2020-12-02 00:00:00.000000,215.60,215.75,210.5000,212.660,6414746,-0.017827
997,2020-12-03 00:00:00.000000,213.33,216.93,213.1100,214.680,6463339,0.009499
998,2020-12-04 00:00:00.000000,214.88,217.28,213.0100,217.235,2118319,0.011901


In [335]:
# View the last 5 rows of the DataFrame.

time_clean=clean_date(pypl_dataframe["time"])
time_clean=pd.Series(time_clean)
time_clean



TypeError: 'float' object is not subscriptable

### Step 3: Using hvPlot, create an interactive visualization for the PYPL daily returns. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.

In [312]:
# Create an interactive visualization with hvplot to plot the daily returns for PYPL.
pypl_dataframe["daily_returns"].hvplot(
    x="time"
    ,title="PYPL Daily Returns"
    ,xlabel= 'Time'
    ,ylabel='Returns'
).opts(
    color='blue')


:Curve   [time]   (daily_returns)

### Step 4: Using hvPlot, create an interactive visualization for the PYPL cumulative returns. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.

In [251]:
# Create an interactive visaulization with hvplot to plot the cumulative returns for PYPL.
(1+pypl_dataframe["daily_returns"]).cumprod().hvplot()

##******FALTA INDICE***


:Curve   [index]   (daily_returns)

## Optimize the SQL Queries

For this part of the assignment, you’ll continue to analyze a single asset (PYPL) from the ETF. You’ll use advanced SQL queries to optimize the efficiency of accessing data from the database.

Complete the following steps:

1. Access the closing prices for PYPL that are greater than 200 by completing the following steps:

1. Access the closing prices for PYPL that are greater than 200 by completing the following steps:

    - Write a SQL `SELECT` statement to select the dates where the PYPL closing price was higher than 200.0.

    - Select the “time” and “close” columns for those dates where the closing price was higher than 200.0.

    - Using the SQL statement, read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.

2. Find the top 10 daily returns for PYPL by completing the following steps:

    -  Write a SQL statement to find the top 10 PYPL daily returns. Make sure to do the following:

        * Use `SELECT` to select only the “time” and “daily_returns” columns.

        * Use `ORDER` to sort the results in descending order by the “daily_returns” column.

        * Use `LIMIT` to limit the results to the top 10 daily return values.

    - Using the SQL statement, read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.


### Step 1: Access the closing prices for PYPL that are greater than 200 by completing the following steps:

    - Write a SQL `SELECT` statement to select the dates where the PYPL closing price was higher than 200.0.

    - Select the “time” and “close” columns for those dates where the closing price was higher than 200.0.

    - Using the SQL statement, read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.


In [122]:
# Write a SQL SELECT statement to select the time and close columns 
# where the PYPL closing price was higher than 200.0.
query = """
SELECT time, close 
FROM PYPL
WHERE close>200

"""

# Using the query, read the data from the database into a Pandas DataFrame
pypl_higher_than_200 = pd.read_sql_query(query, engine)

# Review the resulting DataFrame
pypl_higher_than_200


,time,close
0,2020-08-05 00:00:00.000000,202.920
1,2020-08-06 00:00:00.000000,204.090
2,2020-08-25 00:00:00.000000,201.710
3,2020-08-26 00:00:00.000000,203.530
4,2020-08-27 00:00:00.000000,204.340
5,2020-08-28 00:00:00.000000,204.480
6,2020-08-31 00:00:00.000000,203.950
7,2020-09-01 00:00:00.000000,208.920
8,2020-09-02 00:00:00.000000,210.820
9,2020-09-03 00:00:00.000000,205.070


### Step 2: Find the top 10 daily returns for PYPL by completing the following steps:

    -  Write a SQL statement to find the top 10 PYPL daily returns. Make sure to do the following:

        * Use `SELECT` to select only the “time” and “daily_returns” columns.

        * Use `ORDER` to sort the results in descending order by the “daily_returns” column.

        * Use `LIMIT` to limit the results to the top 10 daily return values.

    - Using the SQL statement, read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.


In [123]:
# Write a SQL SELECT statement to select the time and daily_returns columns
# Sort the results in descending order and return only the top 10 return values
query = """
SELECT time, daily_returns
FROM PYPL
ORDER by daily_returns  desc
LIMIT 10
"""

# Using the query, read the data from the database into a Pandas DataFrame
pypl_top_10_returns = pd.read_sql(query, engine)

# Review the resulting DataFrame
pypl_top_10_returns


,time,daily_returns
0,2020-03-24 00:00:00.000000,0.140981
1,2020-05-07 00:00:00.000000,0.140318
2,2020-03-13 00:00:00.000000,0.138700
3,2020-04-06 00:00:00.000000,0.100877
4,2018-10-19 00:00:00.000000,0.093371
5,2019-10-24 00:00:00.000000,0.085912
6,2020-11-04 00:00:00.000000,0.080986
7,2020-03-10 00:00:00.000000,0.080863
8,2020-04-22 00:00:00.000000,0.075321
9,2018-12-26 00:00:00.000000,0.074656


## Analyze the Fintech ETF Portfolio

For this part of the assignment, you’ll build the entire ETF portfolio and then evaluate its performance. To do so, you’ll build the ETF portfolio by using SQL joins to combine all the data for each asset.

Complete the following steps:

1. Write a SQL query to join each table in the portfolio into a single DataFrame. To do so, complete the following steps:

    - Use a SQL inner join to join each table on the “time” column. Access the “time” column in the `GDOT` table via the `GDOT.time` syntax. Access the “time” columns from the other tables via similar syntax.

    - Using the SQL query, read the data from the database into a Pandas DataFrame. Review the resulting DataFrame.

2. Create a DataFrame that averages the “daily_returns” columns for all four assets. Review the resulting DataFrame.

    > **Hint** Assuming that this ETF contains equally weighted returns, you can average the returns for each asset to get the average returns of the portfolio. You can then use the average returns of the portfolio to calculate the annualized returns and the cumulative returns. For the calculation to get the average daily returns for the portfolio, use the following code:
    >
    > ```python
    > etf_portfolio_returns = etf_portfolio['daily_returns'].mean(axis=1)
    > ```
    >
    > You can use the average daily returns of the portfolio the same way that you used the daily returns of a single asset.

3. Use the average daily returns in the `etf_portfolio_returns` DataFrame to calculate the annualized returns for the portfolio. Display the annualized return value of the ETF portfolio.

> **Hint**  To calculate the annualized returns, multiply the mean of the `etf_portfolio_returns` values by 252.
>
> To convert the decimal values to percentages, multiply the results by 100.

4. Use the average daily returns in the `etf_portfolio_returns` DataFrame to calculate the cumulative returns of the ETF portfolio.

5. Using hvPlot, create an interactive line plot that visualizes the cumulative return values of the ETF portfolio. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.


### Step 1: Write a SQL query to join each table in the portfolio into a single DataFrame. To do so, complete the following steps:

    - Use a SQL inner join to join each table on the “time” column. Access the “time” column in the `GDOT` table via the `GDOT.time` syntax. Access the “time” columns from the other tables via similar syntax.

    - Using the SQL query, read the data from the database into a Pandas DataFrame. Review the resulting DataFrame.

In [136]:
# Create a SQL query to join each table in the portfolio into a single DataFrame 
# Use the time column from each table as the basis for the join
query = """
SELECT  *
FROM GDOT, GS, PYPL, SQ
WHERE GDOT.time=GS.time
AND PYPL.time=SQ.time
AND GDOT.time=PYPL.time
"""
#['GDOT', 'GS', 'PYPL', 'SQ']
# Using the query, read the data from the database into a Pandas DataFrame
etf_portfolio = pd.read_sql_query(query, engine)

# Review the resulting DataFrame
etf_portfolio


,time,GDOT.daily_returns,GS.daily_returns,PYPL.daily_returns,SQ.daily_returns,time,open,high,low,close,...,close,volume,daily_returns,time,open,high,low,close,volume,daily_returns
0,2016-12-16 00:00:00.000000,-0.023218,-0.016708,-0.005564,0.017339,2016-12-16 00:00:00.000000,24.41,24.7300,23.9400,23.980,...,39.320,7298861,-0.005564,2016-12-16 00:00:00.000000,14.29,14.4700,14.2300,14.3750,4516341,0.017339
1,2016-12-19 00:00:00.000000,-0.007923,0.000795,0.003306,-0.001043,2016-12-19 00:00:00.000000,24.00,24.0100,23.5500,23.790,...,39.450,3436478,0.003306,2016-12-19 00:00:00.000000,14.34,14.6000,14.3000,14.3600,3944657,-0.001043
2,2016-12-20 00:00:00.000000,0.001261,0.016602,0.007351,0.009053,2016-12-20 00:00:00.000000,23.75,23.9400,23.5800,23.820,...,39.740,2940991,0.007351,2016-12-20 00:00:00.000000,14.73,14.8200,14.4100,14.4900,5207412,0.009053
3,2016-12-21 00:00:00.000000,0.001679,-0.006911,0.008807,-0.007591,2016-12-21 00:00:00.000000,23.90,23.9700,23.6900,23.860,...,40.090,5826704,0.008807,2016-12-21 00:00:00.000000,14.45,14.5400,14.2701,14.3800,3901738,-0.007591
4,2016-12-22 00:00:00.000000,0.006077,-0.005178,-0.010227,-0.023644,2016-12-22 00:00:00.000000,23.90,24.0100,23.7000,24.005,...,39.680,4338385,-0.010227,2016-12-22 00:00:00.000000,14.33,14.3400,13.9301,14.0400,3874004,-0.023644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2020-11-30 00:00:00.000000,-0.043750,-0.021266,0.013629,-0.007153,2020-11-30 00:00:00.000000,55.87,56.0899,53.1100,53.550,...,214.200,8992681,0.013629,2020-11-30 00:00:00.000000,213.00,213.5400,200.2300,210.9800,10485040,-0.007153
995,2020-12-01 00:00:00.000000,0.004482,0.006549,0.010831,-0.037823,2020-12-01 00:00:00.000000,54.00,54.2500,52.0007,53.790,...,216.520,9148174,0.010831,2020-12-01 00:00:00.000000,212.56,212.9500,200.5000,203.0000,9911165,-0.037823
996,2020-12-02 00:00:00.000000,-0.027328,0.024387,-0.017827,-0.004384,2020-12-02 00:00:00.000000,53.20,53.7900,50.9400,52.320,...,212.660,6414746,-0.017827,2020-12-02 00:00:00.000000,200.13,204.0596,195.1300,202.1100,9968056,-0.004384
997,2020-12-03 00:00:00.000000,0.027523,-0.008959,0.009499,0.016921,2020-12-03 00:00:00.000000,52.48,54.1600,52.0200,53.760,...,214.680,6463339,0.009499,2020-12-03 00:00:00.000000,207.00,208.6900,204.0500,205.5300,6497836,0.016921


### Step 2: Create a DataFrame that averages the “daily_returns” columns for all four assets. Review the resulting DataFrame.

In [143]:
# Create a DataFrame that displays the mean value of the “daily_returns” columns for all four assets.
# Use the time column from each table as the basis for the join
query2 = """
SELECT  GDOT.time, GDOT.daily_returns as 'GDOT.daily_returns',
   GS.daily_returns as 'GS.daily_returns',
   PYPL.daily_returns as 'PYPL.daily_returns',
   SQ.daily_returns as 'SQ.daily_returns'
FROM GDOT, GS, PYPL, SQ
WHERE GDOT.time=GS.time
AND PYPL.time=SQ.time
AND GDOT.time=PYPL.time
"""

#['GDOT', 'GS', 'PYPL', 'SQ']
# Using the query, read the data from the database into a Pandas DataFrame
etf_portfolio2 = pd.read_sql_query(query2, engine)
etf_portfolio2=etf_portfolio2.set_index("time") 
etf_portfolio2

etf_portfolio_returns = etf_portfolio2.mean(axis=1)

# Review the resulting DataFrame
etf_portfolio_returns


time
2016-12-16 00:00:00.000000   -0.007038
2016-12-19 00:00:00.000000   -0.001216
2016-12-20 00:00:00.000000    0.008567
2016-12-21 00:00:00.000000   -0.001004
2016-12-22 00:00:00.000000   -0.008243
                                ...   
2020-11-30 00:00:00.000000   -0.014635
2020-12-01 00:00:00.000000   -0.003990
2020-12-02 00:00:00.000000   -0.006288
2020-12-03 00:00:00.000000    0.011246
2020-12-04 00:00:00.000000    0.009108
Length: 999, dtype: float64

### Step 3: Use the average daily returns in the etf_portfolio_returns DataFrame to calculate the annualized returns for the portfolio. Display the annualized return value of the ETF portfolio.

In [362]:
print(etf_portfolio_returns.index[-1])
print(etf_portfolio_returns.index[0])

period_in_days=datetime.strptime('2020-12-04', "%Y-%m-%d")-datetime.strptime('2016-12-16', "%Y-%m-%d")
period_in_years=period_in_days.dys/365.25
print(f"Period in days is: {period_in_days.days}, which are {period_in_years:,.2f} years")


2020-12-04 00:00:00.000000
2016-12-16 00:00:00.000000
Period in days is: 1449, which are 3.97 years


In [383]:
# Use the average daily returns provided by the etf_portfolio_returns DataFrame 
# to calculate the annualized return for the portfolio. 
annualized_etf_portfolio_returns=etf_portfolio_returns.mean()*252
annualized_etf_portfolio_returns

print(f"An estimation of the annualized return calculated using daily average return times 252 trading days is: {annualized_etf_portfolio_returns*100:,.2f}% ")



An estimation of the annualized return calculated using daily average return times 252 trading days is: 43.83% 


In [387]:
# Use the average daily returns provided by the etf_portfolio_returns DataFrame 
# to calculate the cumulative returns

etf_cumulative_returns = (1+etf_portfolio_returns).cumprod()
annualized_etf_portfolio_returns=etf_cumulative_returns[len(etf_cumulative_returns)-1]**(365.25/period_in_days.days)-1

# Display the annualized return value of the ETF portfolio.
display(f"The actual annualized return of the ETF equally weighted portfolio is {annualized_etf_portfolio_returns*100:,.2f}%")

'The actual annualized return of the ETF equally weighted portfolio is 45.43%'

### Step 4: Use the average daily returns in the `etf_portfolio_returns` DataFrame to calculate the cumulative returns of the ETF portfolio.

In [395]:
# This is above the initial investment (not the final balance, but return)
etf_cumulative_returns = (1+etf_portfolio_returns).cumprod()-1

# Display the final cumulative return value
print(f"The cumulative return of the investment, above the initial balance, not annualized, is of {etf_cumulative_returns[len(etf_cumulative_returns)-1]*100:,.2f}%")


The cumulative return of the investment, above the initial balance, not annualized, is of 341.83%


### Step 5: Using hvPlot, create an interactive line plot that visualizes the cumulative return values of the ETF portfolio. Reflect the “time” column of the DataFrame on the x-axis. Make sure that you professionally style and format your visualization to enhance its readability.

In [340]:
# Using hvplot, create an interactive line plot that visualizes the ETF portfolios cumulative return values.
etf_cumulative_returns["annualized_return"].hvplot(x="time")


:Curve   [time]   (annualized_return)